### まずはある数字の配列からある値を作れるかを判定する関数を作成

In [13]:
from itertools import permutations, product

def can_achieve_value(arr, val):
    operators = ['+', '-', '*', '/']
    n = len(arr)

    # 配列の要素の全ての可能な順列を生成する
    all_permutations = permutations(arr)

    for permutation in all_permutations:
        # それぞれの順列に対して、全ての可能な演算子の組み合わせを試す
        all_operator_combinations = product(operators, repeat=n-1)
        for operator_combination in all_operator_combinations:
            expression = f"{permutation[0]}"
            for i in range(1, n):
                expression += f" {operator_combination[i-1]} {permutation[i]}"
            # 式を評価し、目標の値と一致するかどうかを確認する
            try:
                if eval(expression) == val:
                    return True, expression  # 評価が目標の値と一致する場合、Trueと式を返す
            except ZeroDivisionError:
                continue  # 0で除算するエラーをキャッチし、無視する

    return False, ""  # どの組み合わせも目標の値を達成できない場合、Falseと空の文字列を返す

# 呼び出し例
result, expression = can_achieve_value([3, 8, 3, 8], 24)
print(result, expression)  # 出力: True "1 + 2 + 3 + 4"


False 


上記判定法だと[3,8,3,8]->24(#8 /(3-8/3)=24)が探し出せない
以下改良案(逆ポーランド記法を用いて表現力と一位性を構成)

In [99]:
from itertools import permutations, product
import math

def eval_rpn(expression):
    stack = []
    for token in expression:
        if token in {'+', '-', '*', '/'}:
            # スタックに少なくとも2つの要素があることを確認する
            if len(stack) < 2:
                return None
            b = stack.pop()
            a = stack.pop()
            if token == '+':
                stack.append(a + b)
            elif token == '-':
                stack.append(a - b)
            elif token == '*':
                stack.append(a * b)
            elif token == '/':
                # 0での除算を避ける
                if b == 0:
                    return None
                stack.append(a / b)
        else:
            stack.append(float(token))  # 数字を浮動小数点数として扱う
    # スタックに1つの要素が残っていることを確認する
    return stack[0] if len(stack) == 1 else None

def solve_rpn(nums, target):
    # 数字と演算子の順列を生成
    nums = list(map(lambda num: str(num), nums))
    op_perms = product(['+', '-', '*', '/'], repeat=len(nums)-1)
    
    for op_perm in op_perms:
        for perm in permutations(list(op_perm) + nums):
            # RPN式を評価
            result = eval_rpn(perm)
            if result is not None and abs(result - target) < 1e-8:
                return perm  # RPN式を返す

    return None  # 解が見つからない場合はNoneを返す

# 例の呼び出し
nums = [3, 3, 3, 9]
target = 10
solution = solve_rpn(nums, target)
if solution:
    print(' '.join(map(str, solution)))  # RPN式を出力する
else:
    print("No solution found")

3 3 3 9 / + *


### make10を1~9でできないものがあるかを判定

In [69]:
for i in range(1,10):
    for j in range(i+1,10):
        for k in range(j+1,10):
            for l in range(k+1,10):
                nums = [i,j,k,l]
                solution = solve_rpn(nums, 10)
                if not solution:
                    print(nums, "cannot be solved")


### make kでできるやつをlistUpする

In [67]:
def four_num_check(target):
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    nums = [i,j,k,l]
                    solution = solve_rpn(nums, target)
                    if not solution:
                        return nums
    return None
for target in range(1,25):
    ans = four_num_check(target)
    if not ans:
        print(target)

2
3
4
10


[Twitterに落ちてた疑問](https://x.com/opyunupii/status/1373003757580001280?s=46)を解いてみる
問題は

In [95]:
from tqdm import tqdm
max_ = 0
argmax = []
for i in tqdm(range(1,10)):
    for j in range(i,10):
        for k in range(j,10):
            for l in range(k,10):
                nums = [i,j,k,l]
                for target in range(0,5000):
                    solution = solve_rpn(nums, target)
                    if solution:
                        if(max_ < target):
                            max_ = target
                            argmax = [nums]
                            print(target,nums,solution)
                        elif (max_ == target):
                            argmax.append(nums)
                    else:
                        break
print(max_, argmax)

100%|██████████| 9/9 [00:06<00:00,  1.48it/s]

0 [[1, 1, 1, 1], [1, 1, 1, 3], [1, 1, 2, 2], [1, 1, 2, 4], [1, 1, 3, 3], [1, 1, 3, 5], [1, 1, 4, 4], [1, 1, 4, 6], [1, 1, 5, 5], [1, 1, 5, 7], [1, 1, 6, 6], [1, 1, 6, 8], [1, 1, 7, 7], [1, 1, 7, 9], [1, 1, 8, 8], [1, 1, 9, 9], [1, 2, 2, 3], [1, 2, 2, 5], [1, 2, 3, 4], [1, 2, 3, 6], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 5, 8], [1, 2, 6, 7], [1, 2, 6, 9], [1, 2, 7, 8], [1, 2, 8, 9], [1, 3, 3, 5], [1, 3, 3, 7], [1, 3, 4, 6], [1, 3, 4, 8], [1, 3, 5, 7], [1, 3, 5, 9], [1, 3, 6, 8], [1, 3, 7, 9], [1, 4, 4, 7], [1, 4, 4, 9], [1, 4, 5, 8], [1, 4, 6, 9], [1, 5, 5, 9], [2, 2, 2, 2], [2, 2, 2, 6], [2, 2, 3, 3], [2, 2, 3, 7], [2, 2, 4, 4], [2, 2, 4, 8], [2, 2, 5, 5], [2, 2, 5, 9], [2, 2, 6, 6], [2, 2, 7, 7], [2, 2, 8, 8], [2, 2, 9, 9], [2, 3, 3, 4], [2, 3, 3, 8], [2, 3, 4, 5], [2, 3, 4, 9], [2, 3, 5, 6], [2, 3, 6, 7], [2, 3, 7, 8], [2, 3, 8, 9], [2, 4, 4, 6], [2, 4, 5, 7], [2, 4, 6, 8], [2, 4, 7, 9], [2, 5, 5, 8], [2, 5, 6, 9], [3, 3, 3, 3], [3, 3, 3, 9], [3, 3, 4, 4], [3, 3, 5, 5], [3,

In [109]:
def makebale_list(nums):
    # 数字と演算子の順列を生成
    nums = list(map(lambda num: str(num), nums))
    op_perms = product(['+', '-', '*', '/'], repeat=len(nums)-1)
    made_list = []
    
    for op_perm in op_perms:
        for perm in permutations(list(op_perm) + nums):
            # RPN式を評価
            result = eval_rpn(perm)
            if result is not None and result:
                new = True
                for e in made_list:
                    if(abs(e-result) < 1e-10):
                        new = False
                        break
                if new:
                    made_list.append(result)

    return made_list

### 4則演算によって実現される組み合わせの数をシミュレーション

In [115]:
import random
for i in range(10):
    a,b,c,d = random.random(),random.random(),random.random(),random.random()
    print(len(makebale_list([a,b,c,d])))

1170
1170
1170
1170
1170
1170
1170
1170
1170
1170
